In [1]:
import camelot
import pandas as pd

C:\Users\SaiDheerajPeketi\anaconda3\envs\venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [14]:
pdf_file_name = 'test.pdf'

tables = camelot.read_pdf(pdf_file_name, pages='all', flavor='stream', strip_text='\n')

Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)


In [15]:
print(tables)

<TableList n=6>


In [16]:
for index, table in enumerate(tables):
    table.df.to_csv('table_' + str(index) + '.csv', index=False)